<a href="https://colab.research.google.com/github/SIDHARTH06/ROI_Cutter/blob/master/centernetmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm

     |████████████████████████████████| 431 kB 5.4 MB/s 


In [ ]:
import cv2

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
import timm

import numpy as np

In [ ]:
class downSample(nn.Module):
  def __init(self):
    super(downSample, self).__init__()
    self.ag=nn.AvgPool2d(3,stride=1)
  def forward(self,X):
    X=self.ag(X)
    return X

In [ ]:
class Encoder(nn.Module):
  def __init__(self, pretrained = False, backbone = "resnet34"):
    super(Encoder, self).__init__()

    self.backbone_net = timm.create_model(backbone, pretrained = pretrained)
    

    self.list_layers = list(self.backbone_net.children())[:-3]
    #weight=self.list_layers[0].weight.clone()
    self.list_layers[0]=nn.Conv2d(280,64,(3,3),stride=2,padding=1)

    #self.backbone_net = nn.ModuleList(self.list_layers)




  def forward(self, X):
    outputs = []
    print(self.list_layers[0])
    for i,layer in enumerate(self.list_layers):
      X = layer(X)
      outputs.append(X)

    return outputs[-1]






In [ ]:
class sub_block(nn.Module):
  def __init__(self,ch):
    super(sub_block, self).__init__()

    self.conv1 = nn.Conv2d(ch,ch//4,3, stride = 1, padding=1)
    self.conv2 = nn.Conv2d(ch//4,ch//2,(3,3), stride = 1, padding=1)
    self.usample=nn.Upsample(scale_factor = 2)

  def forward(self, X):
    X=self.conv1(X)
    X=self.conv2(X)
    #print(X.size())
    X=self.usample(X)
    return X

class Decoder(nn.Module):
  def __init__(self, ):
    super(Decoder, self).__init__()
    self.block1=sub_block(256)
    self.block2=sub_block(128)
    self.relu=nn.ReLU()
    self.conv=nn.Conv2d(32,8,(3,3),padding=1,stride=1)
    self.usample1=nn.Upsample(scale_factor = 2)
    self.convpoint=nn.Conv2d(8,6,1)
    self.block3=sub_block(64)
    self.block4=sub_block(32)
  def forward(self, X):
    X=self.block1(X)
    X=self.block2(X)
    X=self.block3(X)
    #X=self.block4(X)
    X=self.conv(X)
    X=self.relu(X)
    X=self.usample1(X)
    #print(X.size())
    X=self.convpoint(X)
    X=self.relu(X)
    return X





In [ ]:
def getheatmap(img,xc,yc,height,width):
  img=((np.exp(-(((np.arange(224)-xc)/(width/10))**2)/2)).reshape(1,-1)*(np.exp(-(((np.arange(224)-yc)/(height/10))**2)/2)).reshape(-1,1))
  return img


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def preprocesslabel():
  path='/content/drive/MyDrive/data/thoracic/Preprocessing/CroppingCode/BoundingBox/spinalBoundfile.txt'
  path2='/content/drive/MyDrive/data/thoracic/Preprocessing/CroppingCode/BoundingBox/ThoraxSpinalBoundBox/'
  inpfie=open(path,'r')
  path3=[]
  lines=inpfie.readlines()
  for line in lines:
    line=line[3:21]
    path3.append(path2+line)
    #print(line)
  inpfie.close()
  path3=path3[24:]
  inputimage=np.zeros((36,280,224,224))
  i=0
  for path in path3:
    path=path+'/img_Bound.npy'
    img=np.load(path)
    img=np.array(img,dtype='float32')
    imge=np.zeros((img.shape[0],224,224),dtype='float32')
    j=0
    for im in img:
      im=cv2.resize(im,(224,224))
      imge[j]=im
      j=j+1
    imge=np.array(imge,dtype=int)
    inputimage[i][:imge.shape[0]]=imge
    i=i+1
  #input
  return inputimage

In [ ]:
input=preprocesslabel()

In [ ]:
input[15].shape

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.encoder=Encoder()
    self.decoder=Decoder()
  
  def forward(self,inp):
    inp=self.encoder(inp)
    inp=self.decoder(inp)
    return inp


In [ ]:
model=Model()

In [ ]:
import torch.optim as optim
los = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
path='/content/drive/MyDrive/data/thoracic/Preprocessing/CroppingCode/BoundingBox/spinalBoundfile.txt'
inpfie=open(path,'r')
lines=inpfie.readlines()
lines
out=[]
for line in lines:
  line=line[55:]
  line=line.split(',',6)
  line=line[:6]
  if line[0][0]==' ':
    line[0]=line[0][1:]
  #print(line)
  outp=np.zeros((6,224,224))
  i=0
  for lin in line:
    lin=(int)(lin)//(512//224)
    li=np.zeros((224,224),float)
    li=getheatmap(li,lin,lin,50,50)
    outp[i]=np.array(li)
    i=i+1
  out.append(outp)
    

In [ ]:
out=np.array(out)
out.shape

In [ ]:
input=np.array(input)
input.shape

In [ ]:
o=out[24:]

In [ ]:
o.shape

In [ ]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

x=input
y=o

tensor_x = torch.Tensor(x) 
tensor_y = torch.Tensor(y)

my_dataset = TensorDataset(tensor_x,tensor_y) 
train_loader = DataLoader(my_dataset,batch_size=10) 

In [ ]:
def cee(out,lab):
  sum=0.0
  X = out
  Y = lab
  n = len(X)
  for i in range(0,n): 
    oupt = model(X[i])
    soft = torch.softmax(oupt, dim=0)
    target = Y[i]
    p = soft[target]  
    sum += torch.log(p)
  return -sum / n

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma = 1.0):
        super(FocalLoss, self).__init__()
        self.gamma = torch.tensor(gamma, dtype = torch.float32)
        self.eps = 1e-6

    def forward(self, input, target):
        probs = torch.sigmoid(input)
        log_probs = -torch.log(probs)
        focal_loss = torch.sum(  torch.pow(1-probs + self.eps, self.gamma).mul(log_probs).mul(target)  , dim=1)
        return focal_loss.mean()

In [ ]:
foc=FocalLoss(1)

In [ ]:
l=nn.MSELoss()

In [ ]:
for epoch in range(20):  
    running_loss = 0.0
    for i,data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = l(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        #if i % 2 == 1:  
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss  :.3f}')
        running_loss = 0.0

print('Finished Training')

In [ ]:
out=model(inputs[0].reshape(1,280,224,224)).detach().numpy()

In [ ]:
plt.imshow(out[0][5])

In [ ]:
plt.imshow()

In [ ]:
plt.imshow(o[0][5])